In [1]:
import trimesh
import pymeshfix
import numpy as np
import datajoint as dj

In [2]:
pinky = dj.create_virtual_module("pinky","microns_pinky")
dj.config["display.limit"] = 20

Connecting celiib@10.28.0.34:3306


In [3]:
new_mesh = trimesh.load_mesh("./dendrite_branches/648518346349478473_Basal_5.off",process=False)

In [4]:
new_mesh.show()

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [5]:
#try importing into trimesh
meshfix = pymeshfix.MeshFix(new_mesh.vertices,new_mesh.faces)

In [6]:
meshfix.mesh

PolyData,Information
N Cells,47338
N Points,23696
X Bounds,"4.074e+05, 4.513e+05"
Y Bounds,"1.546e+05, 1.854e+05"
Z Bounds,"4.134e+04, 8.627e+04"
N Arrays,0


In [7]:
meshfix.mesh.extract_edges()

PolyData,Information
N Cells,29156
N Points,20687
X Bounds,"4.074e+05, 4.513e+05"
Y Bounds,"1.546e+05, 1.854e+05"
Z Bounds,"4.134e+04, 8.627e+04"
N Arrays,0


# TRYING TO PREVENT PYMESHFIX FROM FILTERING AWAY PARTS

In [ ]:
search_key = dict(segmentation=3,segment_id=648518346341395072)
verts,triangles = (pinky.Mesh & search_key).fetch1("vertices","triangles")


In [ ]:
triangles

In [8]:
def make_trimesh_object(verts,triangles):
    new_mesh = trimesh.Trimesh()
    new_mesh.faces = triangles
    new_mesh.vertices = verts
    return new_mesh

In [ ]:
new_mesh = trimesh.Trimesh()
new_mesh.faces = triangles
new_mesh.vertices = verts
new_mesh.show()

In [ ]:
new_fix = pymeshfix.MeshFix(verts,triangles)
new_fix.repair(verbose=True,remove_smallest_components=False)

In [ ]:
"""
Conclusion: Still does remove some of the pieces when processing

"""

display_mesh = trimesh.Trimesh()
display_mesh.vertices = new_fix.v
display_mesh.faces = new_fix.f
display_mesh.show()

In [ ]:
new_fix = pymeshfix.MeshFix(verts,triangles)
new_fix.repair(verbose=True,remove_smallest_components=False,joincomp=True)

display_mesh = trimesh.Trimesh()
display_mesh.vertices = new_fix.v
display_mesh.faces = new_fix.f
display_mesh.show()

# Try and see if joining the mesh that is cut open can be done with pymeshfix

In [ ]:
open_mesh_id = 648518346349513769

open_verts, open_faces = (pinky.Decimation35ExcitatoryStitchedMesh() & "segmentation=3" & "segment_id=648518346349513769").fetch1("vertices","triangles")
open_trimesh = make_trimesh_object(open_verts,open_faces)

In [ ]:
open_trimesh.show()

In [ ]:
# get the pymeshfixed version
open_mesh = pymeshfix.MeshFix(open_verts,open_faces)
open_mesh.repair(verbose=True,joincomp=True,remove_smallest_components=False)

In [ ]:
"""
ran with the remove_smallest_components as False
"""
fixed_open_mesh_2 = make_trimesh_object(open_mesh.v,open_mesh.f)
fixed_open_mesh_2.show()

In [ ]:
"""
ran with the remove_smallest_components as True
"""
fixed_open_mesh = make_trimesh_object(open_mesh.v,open_mesh.f)
fixed_open_mesh.show()

In [ ]:
"""
Conclusion: 
Even the join components of the pymeshfix option wouldn't reconnect this mesh


"""

# Trying to just use the hole patch function of pymeshfix to enable skeletonization

In [9]:
from pymeshfix import _meshfix

In [ ]:
dir(_meshfix.PyTMesh)

In [10]:
import warnings
def clean_from_arrays_celii(v, f, verbose=False, joincomp=False,
                      remove_smallest_components=True,clean=True):
    """
    Performs default cleaning procedure on vertex and face arrays
    Returns cleaned vertex and face arrays
    
    Parameters
    ----------
    v : numpy.ndarray
        Numpy n x 3 array of vertices
    f : numpy.ndarray
        Numpy n x 3 array of faces.
    verbose : bool, optional
        Prints progress to stdout.  Default True.
    joincomp : bool, optional
        Attempts to join nearby open components.  Default False
    remove_smallest_components : bool, optional
        Remove all but the largest isolated component from the mesh
        before beginning the repair process.  Default True.
    Examples
    --------
    >>>
    >>> CleanFromFile('inmesh.ply', 'outmesh.ply')
    """
    # Create mesh object and load from file
    tin = _meshfix.PyTMesh(verbose)
    tin.load_array(v, f)

    # repari and return vertex and face arrays
    repair_celii(tin, verbose, joincomp, remove_smallest_components,clean)
    return tin.return_arrays()

In [11]:
#what the repair function does: 
def repair_celii(tin, verbose=False, joincomp=True, remove_smallest_components=True,clean=True):
    """
    Performs mesh repair using default cleaning procedure using a tin object.
    Internal function.  Use CleanFromFile or CleanFromVF.
    """

    # Keep only the largest component (i.e. with most triangles)
    if remove_smallest_components:
        sc = tin.remove_smallest_components()
        if sc and verbose:
            print('Removed %d small components' % sc)

    # join closest components
    if joincomp:
        tin.join_closest_components()
    
    if tin.boundaries():
        if verbose:
            print('Patching holes...')
        holespatched = tin.fill_small_boundaries()
        if verbose:
            print('Patched %d holes' % holespatched)
    
    
        
    tin.boundaries()
    if clean == True:
        # Perform mesh cleaning
        if verbose:
            print('Fixing degeneracies and intersections')
        result = tin.clean()
    else:
        print("Skipping the degenerative cleaning")
        result = False

    # Check boundaries again
    if tin.boundaries():
        if verbose:
            print('Patching holes...')
        holespatched = tin.fill_small_boundaries()
        if verbose:
            print('Patched %d holes' % holespatched)
    
        if verbose:
            print('Performing final check...')
        if clean == True:
            if verbose:
                print('Fixing degeneracies and intersections')
            result = tin.clean()
        else:
            print("Skipping the degenerative cleaning")
            result = False

    if result:
        warnings.warn('MeshFix could not fix everything')

In [ ]:
search_key = dict(segmentation=3,segment_id=648518346341395072)
verts,triangles = (pinky.Mesh & search_key).fetch1("vertices","triangles")
cleaned_verts, cleaned_triangles= clean_from_arrays_celii(verts,triangles,verbose=True,joincomp=True,remove_smallest_components=False,clean=False)

In [ ]:
cleaned_mesh = make_trimesh_object(cleaned_verts,cleaned_triangles)
cleaned_mesh.show()

In [ ]:
help()

In [ ]:
#see if calcification can work
cleaned_mesh.export("./dendrite_branches/test_neurite.off")
print("ok")

In [ ]:
"""
Conclusion: Using this process the skeleton was able to be processed

"""

# try on more complicated neurite: 

In [12]:
def pymeshfix_no_degenerative(input_file="",input_verts=[],input_faces=[],output_file=""):
    if input_file != "":
        new_mesh = trimesh.load_mesh(input_file,process=False)
        input_verts = new_mesh.vertices
        input_faces = new_mesh.faces
        
    cleaned_verts, cleaned_triangles= clean_from_arrays_celii(input_verts,input_faces,verbose=True,joincomp=True,remove_smallest_components=False,clean=False)
    cleaned_mesh = make_trimesh_object(cleaned_verts,cleaned_triangles)
    if output_file != "":
        cleaned_mesh.export(output_file)
        #add an extra line to the file
        with open(output_file,"a") as file:
            file.write("\n")
    else:
        return cleaned_mesh.vertices,cleaned_mesh.faces
    
    

In [13]:
import subprocess
def meshlab_fix_manifold_path_specific_mls(path_and_filename,segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if path_and_filename[-4:] == ".off":
        path_and_filename = path_and_filename[:-4]
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    #print("input_mesh = " + str(input_mesh))
    #print("output_mesh = " + str(output_mesh))
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    print('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command)
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result


In [14]:
# Keep only the largest component (i.e. with most triangles)
def get_largest_piece(v,f,verbose=True):

    # Create mesh object and load from file
    tin = _meshfix.PyTMesh(verbose)
    tin.load_array(v, f)

    sc = tin.remove_smallest_components()
    if sc and verbose:
        print('Removed %d small components' % sc)
        
    return tin.return_arrays()

In [1]:
import datajoint as dj
pinky = dj.create_virtual_module("pinky","microns_pinky")
segment_id=648518346341395072
segment_id=648518346341351503
segment_id=648518346349506087 #doesn't skeletonize all the way
segment_id=648518346341395072 #works well
segment_id=648518346349467328 #works well
segment_id=648518346349471124 #works well
segment_id=648518346349471629 #works well
segment_id=648518346349507130 #works well
#segment_id=648518346349506493 #works well

search_key = dict(segmentation=3,segment_id=segment_id)

verts,faces = (pinky.Mesh() & search_key).fetch1("vertices","triangles")
starting_mesh = make_trimesh_object(verts,faces)
starting_mesh.show()



Connecting celiib@10.28.0.34:3306


NameError: name 'make_trimesh_object' is not defined

In [16]:
v,f = get_largest_piece(verts,faces)
starting_mesh_largest = make_trimesh_object(v,f)
starting_mesh_largest.show()

Removed 37 small components


In [17]:
starting_mesh.export("./dendrite_branches/starting_mesh.off")
print("hello")

hello


In [ ]:
output_file="./dendrite_branches/current_mesh.off"

pymeshfix_no_degenerative(input_verts = verts,input_faces=faces,
                          output_file=output_file)

meshlab_script = "remove_duplicate_verts.mls"

print("output_file = " + str(output_file))
output_mesh = meshlab_fix_manifold_path_specific_mls(output_file,search_key["segment_id"],meshlab_script)

meshlab_script_2 = "pymesh_fix_substitute.mls"
output_mesh = meshlab_fix_manifold_path_specific_mls(output_mesh,search_key["segment_id"],meshlab_script_2)


print(output_mesh)


Patching holes...
Patched 47 holes
Skipping the degenerative cleaning
output_file = ./dendrite_branches/current_mesh.off
meshlab_script = remove_duplicate_verts.mls
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i ./dendrite_branches/current_mesh.off -o ./dendrite_branches/current_mesh_mls.off -s remove_duplicate_verts.mls
meshlab_script = pymesh_fix_substitute.mls
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i ./dendrite_branches/current_mesh_mls.off -o ./dendrite_branches/current_mesh_mls_mls.off -s pymesh_fix_substitute.mls
./dendrite_branches/current_mesh_mls_mls.off


In [ ]:
import calcification_Module as cm
cm.calcification(output_mesh[:-4])

In [ ]:
#show the output mesh
print("output_mesh = " + str(output_mesh))
output_trimesh = trimesh.load_mesh(output_mesh)
output_trimesh.show()

In [ ]:
def read_skeleton_revised(file_path):
    with open(file_path) as f:
        bones = np.array([])
        for line in f.readlines():
            #print(line)
            line = (np.array(line.split()[1:], float).reshape(-1, 3))
            #print(line[:-1])
            #print(line[1:])

            #print(bones.size)
            if bones.size <= 0:
                bones = np.stack((line[:-1],line[1:]),axis=1)
            else:
                bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
            #print(bones)


    return np.array(bones).astype(float)

In [ ]:
#read in the skeleton files into an array
total_edges = read_skeleton_revised(output_mesh[:-4]+"_skeleton.cgal")
total_edges_stitched = stitch_skeleton_with_degree_check(total_edges)

In [ ]:
#write all of the edges
np.savez("final_skeletons.npz",total_edges_stitched=total_edges_stitched)

In [ ]:
pinky.NeuriteSkeletonStitched() & "largest_mesh_perc<0.6"